In [3]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
train_df = pd.read_csv("D:/PythonSamples/Mercari/train.tsv", sep='\t', header=0,engine="python")


In [1]:
import spacy

nlp = spacy.load('en')
doc1 = nlp(u'Hello hi there!')
doc2 = nlp(u'Hello hi there!')
doc3 = nlp(u'Hey whatsup?')

print(doc1.similarity(doc2)) # 0.999999954642
print(doc2.similarity(doc3)) # 0.699032527716
print(doc1.similarity(doc3))

1.00000008974
0.459109740008
0.459109740008


In [8]:
test_df = pd.read_csv("D:/PythonSamples/Mercari/test.tsv", delimiter="\t", header=0,engine="python")

In [9]:
test_df.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [10]:
test_df.isnull().sum()

test_id                   0
name                      0
item_condition_id         0
category_name          3058
brand_name           295525
shipping                  0
item_description          0
dtype: int64

In [11]:
test_df['price'] = 999999.00 

In [12]:
test_df = test_df[['test_id','name','item_condition_id','category_name','brand_name','shipping','item_description','price']]

In [13]:
test_df['category_name'].fillna('NoValue', inplace=True)

In [14]:
test_df['brand_name'].fillna('NoValue', inplace=True)

In [5]:
sample_sf = pd.read_csv("D:/PythonSamples/Mercari/sample_submission.csv", sep=',', header=0,engine="python")

In [5]:
#len(train_df) --> 1482535
#train_df.isnull().sum()
train_df['price'].min()

0.0

In [15]:
train_df['category_name'].fillna('NoValue', inplace=True)

In [16]:
train_df['brand_name'].fillna('NoValue', inplace=True)

In [18]:
train_df = train_df[train_df['price'] != 0.0]

In [8]:
#train_df['category_name'].value_counts()
#train_df['brand_name'].value_counts()
#train_df.info()

In [19]:
x = train_df[['category_name','name']]
y = test_df[['category_name','name']]

category_df = x
category_df = category_df.append(y)

In [21]:
len(category_df)

2175020

In [22]:
category_df.isnull().sum()

category_name    0
name             0
dtype: int64

In [512]:
#category_df['category_name'].fillna('NoValue', inplace=True)

In [23]:
category_df['name'] = category_df['name'].astype(str).str.split(' ').apply(list)

In [24]:
category_ref = category_df.groupby('category_name')['name'].apply(list)

In [25]:
category_ref.isnull().sum()

0

In [26]:
category_ref_df = pd.DataFrame({'category_name':category_ref.index, 'name':category_ref.values}) 

In [27]:
def flattenlistname(row):
    listoflist = row['name']
    #finallist = reduce(lambda x,y: x+y,listoflist)
    finallist = ' '.join(str(r) for v in listoflist for r in v)
    #print(row['category_name'])
    return finallist

In [28]:
category_ref_df['name'] = category_ref_df.apply(flattenlistname, axis = 1)

In [29]:
category_ref_df['name'] = category_ref_df['name'].str.lower()

In [31]:
def removeDupl(text):
    #print(text)
    list_ = re.sub('[^A-Za-z ]+', '',text)
    list_val = list_.split(' ')
    #print(list_val)
    finallist = list(set(list_val))
    #print(finallist)
    #final = finallist#[re.sub('[^a-zA-Z0-9]+', '', _) for _ in finallist]
    #print(final)
    #final_list = list(filter(None, final))
    final_list1 = []
    for i in finallist:
        #print(i)
        if len(i) > 1:
            final_list1.append(i)
    return final_list1


In [32]:
category_ref_df['name'] = category_ref_df['name'].apply(lambda x: removeDupl(x))

In [630]:
category_ref_df['name'] = category_ref_df['name'].apply(lambda x: cleanUnwantedDesc(x))

In [635]:
#len(category_ref_df)
category_ref_df

,category_name,name
0,Beauty/Bath & Body/Bath,"[shoemama, spongell, soapgreat, extender, rock..."
1,Beauty/Bath & Body/Bathing Accessories,"[women, item, break, hardware, print, cloth, b..."
2,Beauty/Bath & Body/Cleansers,"[crysrockz, grab, halloween, iloveshop, nectar..."
3,Beauty/Bath & Body/Other,"[pregnancy, women, tests, hair, bounty, print,..."
4,Beauty/Bath & Body/Scrubs & Body Treatments,"[skies, game, extender, cleaner, packages, ite..."
5,Beauty/Bath & Body/Sets,"[poshe, scentsy, clearance, hair, castaway, br..."
6,Beauty/Fragrance/Candles & Home Scents,"[grab, ramey, crazycl, packages, rocket, scent..."
7,Beauty/Fragrance/Kids,"[galaxy, swirl, summer, tender, salad, effect,..."
8,Beauty/Fragrance/Men,"[dolcegabbana, ck, women, suits, bijan, brooks..."
9,Beauty/Fragrance/Other,"[euphoria, taylor, sephora, living, bottles, s..."


In [21]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

def cleanUnwanted(row):
    name = row['name']
    name = re.sub('[^A-Za-z ]+', '',name)
    tokens = nltk.word_tokenize(name)
    tokens
    tagged = nltk.pos_tag(tokens)
    cleansed = ''
    for v,k in tagged:
        if k == 'NN' or k == 'NNP' or k == 'NNPS' or k == 'NNS':
            cleansed=cleansed+v+' '
    return cleansed.rstrip()

In [94]:
#df_split1.to_csv("D:/PythonSamples/Mercari/category_ref_df.csv")

In [ ]:
#category_ref_df['cl_name'] = category_ref_df.apply(cleanUnwanted, axis = 1)

In [77]:
category_ref_df.loc[category_ref_df['category_name'] == 'NoValue']

,category_name,name
832,NoValue,"'Bundle', '3', 'Nora', 'Roberts', 'Books', 'AC..."


In [631]:
category_ref_df = category_ref_df[category_ref_df.category_name != 'NoValue']

In [78]:
len(category_ref_df)

1311

In [46]:
#category_ref_df

In [101]:
M = train_df[['category_name','item_description']]
N = test_df[['category_name','item_description']]

category_des_df = M
category_des_df = category_des_df.append(N)

In [102]:
category_des_df.isnull().sum()

category_name       9385
item_description       4
dtype: int64

In [103]:
category_des_df['category_name'].fillna('NoValue', inplace=True)

In [108]:
category_des_df['item_description'] = category_des_df['item_description'].astype(str).str.split(' ').apply(list)

In [234]:
category_des_ref = category_des_df.groupby('category_name')['item_description'].apply(list)

In [222]:
#category_des_ref = category_des_df.groupby('category_name')['item_description'].apply(str)

In [236]:
#category_des_ref

In [527]:
category_des_ref_df = pd.DataFrame({'category_name':category_des_ref.index, 'item_description':category_des_ref.values}) 

In [528]:
type(category_des_ref_df['item_description'])

pandas.core.series.Series

In [529]:
#category_des_ref_df['item_description'] = category_des_ref_df['item_description'].astype(str).str.replace('\[','')
#category_des_ref_df['item_description'] = category_des_ref_df['item_description'].str.replace('\]','')

In [530]:
#category_des_ref_df

In [531]:
from functools import reduce
def flattenlist(row):
    listoflist = row['item_description']
    #finallist = reduce(lambda x,y: x+y,listoflist)
    finallist = ' '.join(str(r) for v in listoflist for r in v)
    #print(row['category_name'])
    return finallist

In [532]:
category_des_ref_df['item_description'] = category_des_ref_df.apply(flattenlist, axis = 1)
#category_des_ref_df['item_description']

In [533]:
category_des_ref_df['item_description'] = category_des_ref_df['item_description'].str.lower()

In [534]:
#test_category_des_ref_df = category_des_ref_df

In [535]:
def removeDup(row):
    list_val = row('item_description')
    list_val = list_val.tolist()
    finallist = set(list_val)
    return finallist

def removeDupl(text):
    #print(text)
    list_ = re.sub('[^A-Za-z ]+', '',text)
    list_val = list_.split(' ')
    #print(list_val)
    finallist = list(set(list_val))
    #print(finallist)
    #final = finallist#[re.sub('[^a-zA-Z0-9]+', '', _) for _ in finallist]
    #print(final)
    #final_list = list(filter(None, final))
    final_list1 = []
    for i in finallist:
        #print(i)
        if len(i) > 1:
            final_list1.append(i)
    return final_list1


In [536]:
category_des_ref_df['item_description'] = category_des_ref_df['item_description'].apply(lambda x: removeDupl(x))

In [365]:
#category_des_ref_df['item_description'] = category_des_ref_df['item_description'].astype(str).str.split(',')#.apply(list)

In [366]:
#category_des_ref_df['item_description'] = category_des_ref_df.apply(removeDup,axis = 1)

In [537]:
def cleanUnwantedDesc(row):
    item_desc = str(row)
    item_desc = re.sub('[^A-Za-z ]+', '',item_desc)
    tokens = nltk.word_tokenize(item_desc)
    tokens
    tagged = nltk.pos_tag(tokens)
    cleansed = ''
    for v,k in tagged:
        if k == 'NN' or k == 'NNP' or k == 'NNPS' or k == 'NNS':
            cleansed=cleansed+v+' '
    return cleansed.rstrip().split()

In [539]:
category_des_ref_df['item_description'] = category_des_ref_df['item_description'].apply(lambda x: cleanUnwantedDesc(x))

In [639]:
category_des_ref_df = category_des_ref_df[category_des_ref_df.category_name != 'NoValue']

In [541]:
#category_des_ref_df#['item_description']

In [508]:
#category_desc_ref_df.to_csv("D:/PythonSamples/Mercari/category_des_ref_df.csv")

In [487]:
#te = train_df[train_df['brand_name'].isnull()]#train_df[train_df['category_name'].isnull()]
#te[te['category_name'].isnull()]
train_df_price = train_df[train_df['category_name'].isnull()]
test_df_price = test_df[test_df['category_name'].isnull()]

In [608]:
test_df_price

,test_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
217,217,Lularoe XS Carly Dress,3,NaN,LuLaRoe,999999.0,0,XS Lularoe Carly Dress. Has peach tone color s...
555,555,Victoria's Secret VerySexy Touch Perfume,2,NaN,Victoria's Secret,999999.0,0,No description yet
702,702,Men's All Black Low Top Converse,3,NaN,Converse,999999.0,0,"Men's Converse, size 11, all black. They have ..."
1110,1110,92 Polly Pocket Jewel Palace/Ice Kingdom,3,NaN,MattelÂ®,999999.0,0,Set of 2. In good condition. Both are complete...
1476,1476,Bundle for Vanessa Wieseler,1,NaN,Avent,999999.0,0,Avent Bottle Warmer Brand New Open Package [rm...
1586,1586,Mens Shake Weight,2,NaN,NaN,999999.0,0,No description yet
1983,1983,Knee Sleeves Workout XS,1,NaN,NaN,999999.0,0,XS brand new in package
2317,2317,Honeywell Money Lockbox,3,NaN,NaN,999999.0,0,No description yet
2376,2376,#84 Wholesale Lot Workout Clothing 30 Pc,3,NaN,NaN,999999.0,0,Bin Pickers from YT wholesale lot #84. All wom...
2508,2508,Hollister sweatpants,3,NaN,Hollister,999999.0,0,"Juniors small in great condition. Very soft, c..."


In [96]:
#category_ref_df['name']

In [498]:
a = [1, 2, 3, 4, 3, 4, 9, 8]
b = [9, 8, 7, 6, 5, 5, 5, 1, 1]
len(list(set(a) & set(b)))


3

In [584]:
b_x = train_df['brand_name']
b_y = test_df['brand_name']

brands = b_x
brands = brands.append(b_y)

In [585]:
brands = pd.DataFrame(brands)#

In [586]:
brands = brands.dropna(axis=0, how='any').drop_duplicates(subset=None, keep='first', inplace=False)

In [587]:
def removeDuplbrand(text):
    #print(text)
    list_ = re.sub('[^A-Za-z ]+', '',text)
    list_val = list_
    return list_val
   

In [590]:
brands['brand_name'] = list(brands['brand_name'].apply(lambda x: removeDuplbrand(x)))

In [592]:
#brands

In [775]:
def findmatch(list_v):
    wval = list_v['category_name']#'Men\'s All Black Low Top Converse'#'Lularoe XS Carly Dress'#'(5) Nike boys t-shirts size 6'##'Bundle for Vanessa Wieseler
    wval = wval.lower().split(' ')
    fnl = {}
    fnl_list = {}
    for index, row in category_ref_df.iterrows():
        fnl[row['category_name']] = str(len(list(set(wval) & set(row['name']))))
    for k,v in fnl.items():
        if v == max(fnl.values()):
            fnl_list[k] = v 
    if len(fnl_list) > 1:
        return 'Other/Other/Other'
    else:
        for ke,va in fnl_list.items():
            return str(ke)

In [703]:
def findmatchdesc(list_v):
    wval = 'Lularoe XS Carly Dress'#'(5) Nike boys t-shirts size 6'#'Lularoe XS Carly Dress'#''#'Bundle for Vanessa Wieseler
    wval = wval.lower().split(' ')
    fnl = {}
    fnl_list = {}
    for index, row in category_des_ref_df.iterrows():
        fnl[row['category_name']] = str(len(list(set(wval) & set(row['item_description']))))
    for k,v in fnl.items():
        if v == max(fnl.values()):
            fnl_list[k] = v 
    return fnl_list

In [773]:
list_v = ''
result_df = findmatch(list_v)#)

In [704]:
result_df = findmatchdesc(category_des_ref_df)#)

In [774]:
result_df

'Other/Other/Other'

In [755]:
com_list = []
searchval = 'Hollister sweatpants'#'Men\'s All Black Low Top Converse'#'Lularoe XS Carly Dress'
for k,v in result_df.items():
    com_list.append(k)

categ_list = []

for i in com_list:
    categ = i+' '+searchval
    categ = categ.replace('/',' ').lower()
    categ = re.sub('[^A-Za-z ]+', '',categ)
    categ = categ.replace('\n','')
    categ = categ.split(' ')
    categ_list.append(categ)

new_fnl ={}

for ol in categ_list:
    print(ol)
    for index, row in category_des_ref_df.iterrows():
        new_fnl[row['category_name']] = str(len(list(set(ol) & set(row['item_description']))))

for k,v in new_fnl.items():
    if v == max(new_fnl.values()):
        print(k,v) 
        

['kids', 'boys', '', 'bottoms', 'hollister', 'sweatpants']
['kids', 'boys', '', 'coats', '', 'jackets', 'hollister', 'sweatpants']
['kids', 'girls', '', 'jeans', 'hollister', 'sweatpants']
['kids', 'girls', '', 'other', 'hollister', 'sweatpants']
['kids', 'girls', '', 'tops', '', 'tshirts', 'hollister', 'sweatpants']
['men', 'athletic', 'apparel', 'pants', 'hollister', 'sweatpants']
['men', 'athletic', 'apparel', 'shirts', '', 'tops', 'hollister', 'sweatpants']
['men', 'athletic', 'apparel', 'shorts', 'hollister', 'sweatpants']
['men', 'athletic', 'apparel', 'tracksuits', '', 'sweats', 'hollister', 'sweatpants']
['men', 'jeans', 'classic', 'straight', 'leg', 'hollister', 'sweatpants']
['men', 'pants', 'casual', 'pants', 'hollister', 'sweatpants']
['men', 'pants', 'other', 'hollister', 'sweatpants']
['men', 'sweats', '', 'hoodies', 'hoodie', 'hollister', 'sweatpants']
['men', 'sweats', '', 'hoodies', 'other', 'hollister', 'sweatpants']
['men', 'sweats', '', 'hoodies', 'track', '', 'swea

In [748]:
tcateg_list = [['men', 'mens', 'accessories', 'backpacks', 'bags', '', 'briefcases', 'mens', 'all', 'black', 'low', 'top', 'converse'],
['men', 'mens', 'accessories', 'hats', 'mens', 'all', 'black', 'low', 'top', 'converse'],
['men', 'shoes', 'athletic', 'mens', 'all', 'black', 'low', 'top', 'converse'],
['men', 'shoes', 'fashion', 'sneakers', 'mens', 'all', 'black', 'low', 'top', 'converse'],
['men', 'sweats', '', 'hoodies', 'hoodie', 'mens', 'all', 'black', 'low', 'top', 'converse']]

tnew_fnl = {}
for ol in tcateg_list:
    print(ol)
    for index, row in category_des_ref_df.iterrows():
        tnew_fnl[row['category_name']] = str(len(list(set(ol) & set(row['item_description']))))

for k,v in tnew_fnl.items():
    if v == str(int(max(tnew_fnl.values()))-1):
        print(k,v)         

['men', 'mens', 'accessories', 'backpacks', 'bags', '', 'briefcases', 'mens', 'all', 'black', 'low', 'top', 'converse']
['men', 'mens', 'accessories', 'hats', 'mens', 'all', 'black', 'low', 'top', 'converse']
['men', 'shoes', 'athletic', 'mens', 'all', 'black', 'low', 'top', 'converse']
['men', 'shoes', 'fashion', 'sneakers', 'mens', 'all', 'black', 'low', 'top', 'converse']
['men', 'sweats', '', 'hoodies', 'hoodie', 'mens', 'all', 'black', 'low', 'top', 'converse']
Kids/Boys (4+)/Other 5
Kids/Boys (4+)/Shoes 5
Kids/Boys 0-24 Mos/Shoes 5
Kids/Boys 2T-5T/Bottoms 5
Kids/Boys 2T-5T/Shoes 5
Kids/Boys 2T-5T/Tops & T-Shirts 5
Kids/Girls (4+)/Coats & Jackets 5
Kids/Girls (4+)/Other 5
Kids/Girls 0-24 Mos/Tops & T-Shirts 5
Kids/Girls 2T-5T/Bottoms 5
Kids/Toys/Action Figures & Statues 5
Men/Athletic Apparel/Jackets 5
Men/Coats & Jackets/Vest 5
Men/Coats & Jackets/Windbreaker 5
Men/Men's Accessories/Hats 5
Men/Sweats & Hoodies/Hoodie 5
Men/Tops/Other 5
Men/Tops/T-shirts 5
Men/Tops/Tank 5
Other/Ot

In [759]:
for k,v in new_fnl.items():
    if v == str(int(max(new_fnl.values()))-2):
        print(k,v)

Beauty/Fragrance/Women 5
Beauty/Makeup/Eyes 5
Beauty/Makeup/Face 5
Beauty/Makeup/Nails 5
Electronics/Cell Phones & Accessories/Cases, Covers & Skins 5
Handmade/Clothing/Costume 5
Handmade/Clothing/Lingerie 5
Home/Seasonal DÃ©cor/Halloween 5
Kids/Boys (4+)/Coats & Jackets 5
Kids/Boys (4+)/Top & T-shirts 5
Kids/Boys 0-24 Mos/One-Pieces 5
Kids/Boys 2T-5T/Shoes 5
Kids/Girls (4+)/Coats & Jackets 5
Kids/Girls (4+)/Jeans 5
Kids/Girls (4+)/Other 5
Kids/Girls 0-24 Mos/One-Pieces 5
Kids/Girls 2T-5T/One-Pieces 5
Kids/Girls 2T-5T/Shoes 5
Kids/Girls 2T-5T/Tops & T-Shirts 5
Men/Athletic Apparel/Shirts & Tops 5
Men/Athletic Apparel/Shorts 5
Men/Men's Accessories/Hats 5
Men/Sweaters/Crewneck 5
Men/Sweats & Hoodies/Track & Sweat Pants 5
Sports & Outdoors/Exercise/Fitness accessories 5
Vintage & Collectibles/Antique/Collectibles 5
Women/Athletic Apparel/Sports Bras 5
Women/Coats & Jackets/Other 5
Women/Coats & Jackets/Windbreaker 5
Women/Dresses/Full-Length 5
Women/Dresses/Knee-Length 5
Women/Dresses/Mi